In [1]:
import numpy as np
import pandas as pd
import sys
import re
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
model_path = '~/Git/Titanic/Titanic_model'

/home/lambert/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
import re
def get_title(name):
    if pd.isnull(name):
        return 'Null'
    title_search = re.search('([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1).lower()
    else:
        return 'None'

In [4]:
titles = {'mr':1, 'mrs':2, 'mme':2, 'ms':3, 'miss':3, 'mlle':3, 'don':4, 'sir':4, 'jonkheer':4,
          'major':4, 'col':4, 'dr':4, 'master':4, 'capt':4, 'dona':5, 'lady':5, 'countess':5,
         'rev':6}
df_train['Title'] = df_train['Name'].apply(lambda name: titles.get(get_title(name)))
df_train['Honor'] = df_train['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)
df_test['Title'] = df_test['Name'].apply(lambda name: titles.get(get_title(name)))
df_test['Honor'] = df_test['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)

In [5]:
# 1. Data clearning
# df_train.Age.fillna(fill_df_Age, inplace=True)
# df_test.Age.fillna(fill_df_Age, inplace=True)
for i in range(1,7):
    a = df_train[(df_train.Age.isnull()) & (df_train.Title==i)]
    df_train.Age.iloc[a.index] = df_train.Age[df_train.Title==i].median()
    b = df_test[(df_test.Age.isnull()) & (df_test.Title==i)]
    df_test.Age.iloc[b.index] = df_train.Age[df_train.Title==i].median()

# Cabin - fill nan with 'fill' and select first cabin
df_train.Cabin.fillna('fill',inplace=True)
df_train.Cabin = df_train.Cabin.map(lambda x:x.split(' ')[0])
df_test.Cabin.fillna('fill', inplace=True)
df_test.Cabin = df_test.Cabin.map(lambda x:x.split(' ')[0])

# Sex - male: 1, female: 1
df_train.Sex.replace({'male':1, 'female':0}, inplace=True)
df_test.Sex.replace({'male':1, 'female':0}, inplace=True)

df_train = df_train.dropna(axis=0, how='any')
df_test.Fare.fillna(df_test.Fare.mean(), inplace=True)

# Create a 'Deceased' column for sconed class
df_train['Deceased'] = df_train.Survived.apply(lambda s: int(not s))

/home/lambert/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
# 2. Select the features
sFeatures = ['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',             
 'Cabin',             
 'Embarked',
 'Title',
 'Honor']

X = df_train[sFeatures]
y = df_train[['Survived','Deceased']]
y = y.values.reshape(len(y),2)
X_test = df_test[sFeatures]

In [7]:
# 3. Label Encoder
# X_test has elements that X doesn't have
# create a Cabin_labels cover X and X_test
X_Cabin_unique = X['Cabin'].unique()
X_lack = X_test.Cabin[X_test['Cabin'].isin(X_Cabin_unique)==0].values
Cabin_labels = np.append(X_Cabin_unique, X_lack)
# print(X_Cabin_unique.shape, X_Cabin_unique)
# print(Cabin_labels.shape, Cabin_labels)

le_Cabin = LabelEncoder().fit(Cabin_labels)
le_Embarked = LabelEncoder().fit(np.array(X['Embarked'].tolist()))

X.loc[:,'Cabin'] = le_Cabin.transform(X['Cabin'])
X.loc[:,'Embarked'] = le_Embarked.transform(X['Embarked'])
X_test.loc[:,'Cabin'] = le_Cabin.transform(X_test['Cabin'])
X_test.loc[:,'Embarked'] = le_Embarked.transform(X_test['Embarked'])
X = X.astype(np.float32)
y = y.astype(np.float32)

/home/lambert/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
# Validation
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [9]:
X_val[0:10]

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title,Honor
28,3.0,0.0,21.0,0.0,0.0,7.879200,180.0,1.0,3.0,0.0
178,2.0,1.0,30.0,0.0,0.0,13.000000,180.0,2.0,1.0,0.0
614,3.0,1.0,35.0,0.0,0.0,8.050000,180.0,2.0,1.0,0.0
428,3.0,1.0,30.0,0.0,0.0,7.750000,180.0,1.0,1.0,0.0
325,1.0,0.0,36.0,0.0,0.0,135.633301,83.0,0.0,3.0,0.0
713,3.0,1.0,29.0,0.0,0.0,9.483300,180.0,2.0,1.0,0.0
161,2.0,0.0,40.0,0.0,0.0,15.750000,180.0,2.0,2.0,0.0
785,3.0,1.0,25.0,0.0,0.0,7.250000,180.0,2.0,1.0,0.0
486,1.0,0.0,35.0,1.0,0.0,90.000000,112.0,2.0,2.0,0.0
378,3.0,1.0,20.0,0.0,0.0,4.012500,180.0,0.0,1.0,0.0


### Tensor Graph

In [10]:
def next_batch(batch, X, y):
    num = np.arange(len(X))
    np.random.shuffle(num)
    select = num[0:batch]
    x_batch = X.iloc[select]
    y_batch = y[select]
    
    return(x_batch, y_batch)

In [11]:
batch_size = 200

def optimize(num_iterations):
    total_loss = 0
    for i in range(num_iterations):
        x_batch, y_batch = next_batch(batch_size, X_train_data, y_train_data)
        feed_dict_train = {X_place: x_batch, y_place: y_batch}
        _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
        total_loss += loss
        print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
    print ('Training complete!')

In [12]:
# Tensorflow variables
features= len(sFeatures)
X_place = tf.placeholder(tf.float32, shape=[None,features])
y_place = tf.placeholder(tf.float32, shape=[None,2])
W = tf.Variable(tf.random_normal([features, 2]), name='weights')
b = tf.Variable(tf.zeros(2), name='bias')
logits = tf.matmul(X_place, W) + b
y_pred = tf.nn.softmax(logits)

In [13]:
# Cross Entropy
learning_rate = 0.001
cross_entropy = -tf.reduce_sum(y_place * tf.log(y_pred + 1e-10), axis=1)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_train)
cost = tf.reduce_mean(cross_entropy)
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
saver = tf.train.Saver()

In [14]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

In [15]:
# # training starting option 1
# for epoch in range(10000):
#     total_loss = 0.
#     x_batch, y_batch = next_batch(batch_size, X_train, y_train)
#     feed_dict_train = {X_place: x_batch, y_place: y_batch}
#     _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
#     total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [16]:
# sess.close()

In [18]:
# training starting option 1 with CPU
# for epoch in range(100):
#     total_loss = 0.
#     for i in range(len(X_train)):
#         feed = {X_place: X_train, y_place: y_train}
#         _, loss = sess.run([train_op, cost], feed_dict=feed)
#         total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [ ]:
# training starting option 1 with GPU
# with tf.device('/device:GPU:0'):
start = time.time()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device('/device:GPU:0'):
        

end = time.time()
print('time spend: {:.4f} sec'.format(end-start))

In [ ]:
save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
# sess.close()

In [ ]:
# evaluate the training data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_train = {X_place: X_train_data}
    prediction = sess.run(y_pred, feed_dict=feed_dict_train)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train_data,axis=1))
# evaluate the accuracy
accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

In [ ]:
# evaluate the validate data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_valid = {X_place: X_val_data}
    prediction = sess.run(y_pred, feed_dict=feed_dict_valid)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_val_data,axis=1))
# evaluate the accuracy

accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

In [ ]:
# prediction for test data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_test = {X_place: X_test}
    prediction = sess.run(y_pred, feed_dict=feed_dict_test)
    y_predict = np.argmax(prediction, axis=1)
# evaluate the accuracy

In [ ]:
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_random.csv')

In [ ]:
y_predict.shape

In [ ]:
# print_accuracy()
# plot_example_errors()
# optimize(num_iterations=10)
print_accuracy()
# plot_example_errors()
# plot_weights()

In [ ]:
File_Writer=tf.summary.FileWriter(''/Users/lambert/Documents/Python_code/tensorflow_code/Titanic'
                                  ,sess.graph)

In [ ]:
# with tf.device('/device:GPU:0'):
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:

In [ ]:
# allocate as much GPU memory based on runtime allocations
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
# allocate 40% memory of GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config, ...)

In [ ]:
# Automatically choose existing in case specified one doesn't exit
# allow_soft_placement=True
with tf.device('/device:GPU:2'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with allow_soft_placement and log_device_placement set
# to True.
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True))

In [ ]:
sess.close()

In [ ]:
sys.argv[2]

In [ ]:
device_lib.list_local_devices()

Logistic Regression

In [ ]:
# Logistic Regression
lr = LogisticRegression(C=1, max_iter=100, solver='lbfgs').fit(X_train, y_train)
yp = lr.predict(X_train)
y_predict = lr.predict(X_test)

passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_knn.csv')
# passengerid = np.array(df_train.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': yp}).set_index('PassengerId')
# submission.to_csv('y_train_predict.csv')

Gradient Boosting Classifier

In [ ]:
# Gradient Boosting Classifier
# gbdt = GradientBoostingClassifier(n_estimators=300, max_depth=5).fit(X_train, y_train)
# y_predict = gbdt.predict(X_test)

# passengerid = np.array(df_test.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
# submission.to_csv('submission.csv')

Random forests

In [ ]:
# RandomForest
rfc = RandomForestClassifier(n_estimators=200, max_depth=3).fit(X_train, y_train)
y_predict = rfc.predict(X_test)
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_random.csv')

Stochastic Gradient Descent (SDG)

In [ ]:
# Stochastic Gradient Descent (SGD) easily scale with more than 10^5 samples and features
sgd = SGDClassifier(alpha=0.01, max_iter=1000).fit(X_train, y_train)
y_predict = sgd.predict(X_test)
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_sdg.csv')